In [2]:
!curl https://www.google.com

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="zh-HK"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="mAGoAkIyJ7bCuqauHtAzhw">(function(){var _g={kEI:'cxHIaKm5Nrv50PEP54mUgQw',kEXPI:'0,202854,2,83,4037106,14112,34679,390923,290044,5242138,27,172,36811987,25228681,138268,71240,8041,6754,23879,9138,4600,328,6225,37826,26339,15048,49,15586,30376,28339,42883,5407,5916,353,4100,14780,7758,5826,5774,8976,18635,3927,7,12590,6251,35,3420,2115,712,10656,9756,2352,5683,3605,7329,10442,1515,16484,1281,676,14645,550,6022,1,9634,1033,2294,1535,5083,2,5306,1959,2747,2,4,1,320,842,2,979,2270,1010,667,91,12407,519,4035,10588,4425,1280,591,4,1378,32,104,3355,196,861,36,2803,2114,2,1347,5,683,127,1688,1675,474,16,237,840,766,10,552,4,5,1119,1500,1931,1738,1205,3255,1129,2,876,22,1916,291,547,22,3342,585,466,127,19,3007,26

In [3]:
!nvidia-smi

Mon Sep 15 21:15:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.144                Driver Version: 570.144        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        Off |   00000000:01:00.0 Off |                  N/A |
|  0%   34C    P8             26W /  575W |       0MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
#!pip install transformers datasets evaluate accelerate

In [5]:
from datasets import load_dataset

# SST-2 是 GLUE 基准测试的一部分
dataset = load_dataset("glue", "sst2")

# 打印出来看看数据集的结构
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [6]:
from transformers import AutoTokenizer

# 加载和 BERT-base-uncased 模型配套的 Tokenizer
# 'uncased' 的意思是它会先把所有字母转为小写
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 定义一个函数来处理数据集
# 它会接收一批数据，然后用 tokenizer 对 'sentence' 字段进行编码
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

# 使用 .map() 方法将这个函数应用到整个数据集上
# batched=True 表示一次性处理一批数据，速度更快
tokenized_datasets = dataset.map(tokenize_function, batched=True)

print("预处理完成！")

预处理完成！


In [7]:
from transformers import AutoModelForSequenceClassification

# 加载 BERT 模型用于序列分类
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import AutoModelForSequenceClassification

# 加载 BERT 模型用于序列分类
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments

# 定义训练参数
# "test_trainer" 是输出结果的文件夹名
training_args = TrainingArguments(
    output_dir="my_bert_sst2_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,

    # --- 日志策略 (用于平滑训练曲线) ---
    logging_strategy="steps",
    logging_steps=50,                 # 非常频繁地记录训练日志 -> wandb的【训练损失】曲线会很平滑

    # --- 评估与保存策略 (用于找到最佳模型并绘制验证曲线) ---
    eval_strategy="steps",
    save_strategy="steps",            # !! 保持评估和保存策略一致
    
    # 每200步评估和保存一次。这个频率比日志低，可以节省时间，但又足够密集，
    # 足以找到最佳模型，并为wandb的【验证曲线】提供足够的数据点。
    eval_steps=200,
    save_steps=200,                   # !! 保持评估和保存步数一致

    # --- 加载最佳模型 ---
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    
    # (可选但推荐) 为了防止硬盘被占满，只保留最近的3个检查点
    save_total_limit=3,
    
    report_to="wandb", # 明确指定向wandb报告
)


In [10]:
import numpy as np
import evaluate

# 加载 accuracy 评估指标
metric = evaluate.load("accuracy")

# 定义一个函数，告诉 Trainer 如何计算指标
# 这个函数会在每次评估时被调用
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

print("评估函数定义完成！")

评估函数定义完成！


In [11]:
from transformers import Trainer

# 创建 Trainer 实例
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: maniaamaeovo (maniaamaeovo-mania) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
200,0.235400,0.226052,0.910550
400,0.203300,0.237094,0.901376
600,0.167000,0.225710,0.917431
800,0.168900,0.198868,0.924312
1000,0.150600,0.194370,0.918578
1200,0.085800,0.222667,0.916284
1400,0.087300,0.243116,0.918578
1600,0.094800,0.228022,0.924312
1800,0.087100,0.237748,0.930046
2000,0.102500,0.214646,0.927752


TrainOutput(global_step=3159, training_loss=0.11688037768086515, metrics={'train_runtime': 2110.5255, 'train_samples_per_second': 95.733, 'train_steps_per_second': 1.497, 'total_flos': 5.316079940232192e+16, 'train_loss': 0.11688037768086515, 'epoch': 3.0})